<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Importing-daily-timeseries-Global-dataset" data-toc-modified-id="Importing-daily-timeseries-Global-dataset-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Importing daily timeseries Global dataset</a></span></li><li><span><a href="#Droping-Unnecessary-column-from-dataset" data-toc-modified-id="Droping-Unnecessary-column-from-dataset-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Droping Unnecessary column from dataset</a></span></li><li><span><a href="#Grouping-the-data-by-country" data-toc-modified-id="Grouping-the-data-by-country-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Grouping the data by country</a></span></li></ul></li><li><span><a href="#Prediction-Function" data-toc-modified-id="Prediction-Function-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prediction Function</a></span><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#ARIMA" data-toc-modified-id="ARIMA-1.0.0.1"><span class="toc-item-num">1.0.0.1&nbsp;&nbsp;</span>ARIMA</a></span></li></ul></li></ul></li><li><span><a href="#Using-Interactive-Ipython-widgets" data-toc-modified-id="Using-Interactive-Ipython-widgets-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Using Interactive Ipython widgets</a></span></li></ul></li></ul></div>

# Milestone 3 - Time Series Forecasting

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML
import plotly.express as px
import plotly.tools as tls
import folium
import plotly.graph_objects as go
import ipywidgets as widgets
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import datetime
from dateutil.parser import parse as  date_parser
import numpy as np
import pandas as pd
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import threading
import time
import warnings
warnings.simplefilter('ignore')
%matplotlib inline

#### Importing daily timeseries Global dataset
  - Using pandas read_csv function to get dataset 

In [2]:
df = pd.read_csv (r'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/1/21,4/2/21,4/3/21,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,56517,56572,56595,56676,56717,56779,56873,56943,57019,57144
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,125506,125842,126183,126531,126795,126936,127192,127509,127795,128155
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,117304,117429,117524,117622,117739,117879,118004,118116,118251,118378
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,12053,12115,12174,12231,12286,12328,12363,12409,12456,12497
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,22399,22467,22579,22631,22717,22885,23010,23108,23242,23331


#### Droping Unnecessary column from dataset
  - Using df.drop to drop the columns

In [3]:
df = df.drop(columns=['Province/State','Lat','Long'])

#### Grouping the data by country
  - Using df.groupby to group the cells by country and perform sum

In [4]:
agg_df = df.groupby(['Country/Region']).sum().copy()

In [5]:
countries=list(agg_df.index.values)
countries.insert(0,'World')

In [13]:
countries

['World',
 'Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Diamond Princess',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',


### Prediction Function 
  - Country is provided as Argument
  - Train Test split is performed in the ratio 95:5 respectively
  ###### ARIMA
      - In arima function first the p,d,q are set from a range 0 to 8 
      - Best value of pdq is chosen by iterating and performing the model and comparing the results
      - The best value of pdq is supplied to the model
      - plot_predict() is used to forecast the value of confirmed cases in the coming month
      - Plot are shown using Plotly

In [6]:

def prediction_plots(country):

    def data(country):
        try:
            if(country=='World'):
                data=agg_df.sum(axis = 0, skipna = True)
                data=pd.DataFrame(data)
                data.columns=['World']
            else:
                data=agg_df.loc[country]
                data=pd.DataFrame(data)
        except:
            pass
        return data


    def split(ts):
        #splitting 95%/5% because of little amount of data
        size = int(len(ts) * 0.95)
        train= ts[:size]
        test = ts[size:]
        return(train,test)


    #Arima modeling for ts
    def arima(ts,test):
        
        # define parameters
        p=d=q=range(0,8)
        a=99999
        pdq=list(itertools.product(p,d,q))
        
        # PROGRESS BAR WIDGET
        
        progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Loading: ')
        finished = False
        
        def work(progress):
            total = len(pdq)
            for i in range(total):
                if finished != True:
                    time.sleep(0.2)
                    progress.value = float(i+1)/len(pdq)
                else:
                    progress.value = len(pdq)
                    break

        thread = threading.Thread(target=work, args=(progress,))
        display(progress)
        # Start the progress bar thread
        thread.start()

        # Determining the best parameters
        for var in pdq:
            
            try:
                
                model = ARIMA(ts, order=var)
                result = model.fit()
                
                if (result.aic<=a) :
                    a=result.aic
                    param=var
                    
            except:
                continue
                
        model = ARIMA(ts, order=param)
        result = model.fit()
        finished = True

        # Forecast CURVE
        fig = result.plot_predict(start=int(len(ts) * 0.5), end=int(len(ts) * 1.2))
        fig.set_size_inches(12, 6)
        ax_list = fig.axes
        pred = result.forecast(steps=len(test))[0]    
        
        
        # TEST VS PREDICTION Curve
        plotly_figs = go.Figure()
        plotly_figs.add_trace(go.Scatter(x=test.index.values, y=test[country],
                    mode='lines',
                    name='real values'))
        plotly_figs.add_trace(go.Scatter(x=test.index.values, y=pred,
                    mode='lines',
                    name='predictions'))
        
        
        plotly_figs.update_layout(title="Prediction : COVID 19 cases of " + country, xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800, showlegend=True
        )
        plotly_figs.show()

    train,test=split(data(country))

    arima(train,test)

    

#### Using Interactive Ipython widgets
  - Ipython widgets are used to make the response of arima function responsive according to user input.
  - Progress bar shows the progress of training process.

In [7]:
interact(prediction_plots, country=countries)
fig = go.FigureWidget( layout=go.Layout() )
ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Dropdown(description='country', options=('World', 'Afghanistan', 'Albania', 'Algeria', '…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …